In [1]:
import pandas as pd
import pod5 as p5

def load_and_merge_data(tsv_file, pod_file):
    """Læser tsv- og pod5-filer, og returnerer en flettet DataFrame."""
    polyA_df = pd.read_csv(tsv_file, sep='\t')

    with p5.Reader(pod_file) as reader:
        pod_data = [(str(read.read_id), read.signal) for read in reader.reads()]

    pod_df = pd.DataFrame(pod_data, columns=["read_id", "signal"])
    return polyA_df.merge(pod_df, on="read_id")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_polyA_signals_to_pdf(df, output_filename, max_plots, max_length):
    """Plotte polyA-signaler og gemme dem i en PDF."""
    num_plots = min(len(df), max_plots)
    rows, cols = 5, 3
    pages = (num_plots + (rows * cols) - 1) // (rows * cols)

    global_max_length = 0

    with PdfPages(output_filename) as pdf:
        for page in range(pages):
            fig, axes = plt.subplots(rows, cols, figsize=(15, 9))
            fig.suptitle(f'PolyA Signals (Page {page+1})', fontsize=16)
            axes = axes.flatten()

            for i in range(rows * cols):
                idx = page * (rows * cols) + i
                if idx >= num_plots:
                    axes[i].axis('off')
                    continue

                signal_slice = df['signal'][idx][df['start'][idx]:df['end'][idx]]
                max_length_sliced = len(signal_slice)
                global_max_length = max(global_max_length, max_length_sliced)

                axes[i].plot(signal_slice)
                axes[i].set_title(f'Row {idx+1}')
                axes[i].set_xlim(-100, max_length + 100)
                axes[i].set_ylim(-10, 1000)

            plt.tight_layout(rect=[0, 0, 1, 0.96])
            pdf.savefig(fig)
            plt.close(fig)

    print(f"PDF saved as '{output_filename}'")
    print(f"Max signal length: {global_max_length}")

In [12]:
df = load_and_merge_data("a120_2mod_polyA_position.tsv", "egfp_a120_2mod_polya_reads.pod5")

# Assuming df is your merged DataFrame containing the polyA signals
signal_data = df['signal'][0]  # Extract the first signal

# Find the start and end indices for the first signal (you might already have these in 'start' and 'end' columns)
start_idx = df['start'][0]
end_idx = df['end'][0]

# Slice the signal data for the first signal
signal_slice = signal_data[start_idx:end_idx]

def middle_50_mean(vector):
        n = len(vector)
        start = int(n * 0.25)  # Removes first 25%
        end = int(n * 0.75)    # Removes last 25%
        return np.mean(vector[start:end]), len(vector[start:end])  # Mean of middle 50%


# Calculate the mean of the middle 50%
mean_middle_50, len_50 = middle_50_mean(signal_slice)

# Calculate the median of the entire signal
median_all_values = np.median(signal_slice)

mean_all_values = np.mean(signal_slice)

# Print the results
print(f"Længde af hele data: {len(signal_slice)}")
print(f"Længde af 50% af data: {len_50}")
print(f"Mean of the middle 50%: {mean_middle_50}")
print(f"Median of all values: {median_all_values}")
print(f"Mean of all values: {mean_all_values}")

Længde af hele data: 1296
Længde af 50% af data: 648
Mean of the middle 50%: 494.47067901234567
Median of all values: 495.0
Mean of all values: 494.52314814814815


In [15]:
data_a120_2mod = load_and_merge_data("a120_2mod_polyA_position.tsv", "egfp_a120_2mod_polya_reads.pod5")
plot_polyA_signals_to_pdf(data_a120_2mod, "raw_polyA_signals_a120_2mod.pdf", 60, 11508)

PDF saved as 'raw_polyA_signals_a120_2mod.pdf'
Max signal length: 11508


In [16]:
data_a120_1mod = load_and_merge_data("a120_1mod_polyA_position.tsv", "egfp_a120_1mod_polya_reads.pod5")
plot_polyA_signals_to_pdf(data_a120_1mod, "raw_polyA_signals_a120_1mod.pdf", 60, 3824)

PDF saved as 'raw_polyA_signals_a120_1mod.pdf'
Max signal length: 3824
